<h1 style="text-align:center;">Gathering ESKM Data for HPE iLO Systems</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook demonstrates an example of collecting ESKM (Enterprise Secure Key Manager) data for HPE iLO (Integrated Lights-Out) systems. It utilizes the HPE iLOrest library to interact with the iLO API and retrieve the necessary information.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. The purpose of the get_ESKM function is to retrieve Enterprise Secure Key Manager (ESKM) data from an HPE iLO (Integrated Lights-Out) system. It requires a connection to the iLO system through the Redfish API.</h3><br>
  Here's a step-by-step breakdown of what the function does:
<br><ul>
<li>It starts by initializing a variable called security_service_eskm_uri, which will store the URI (Uniform Resource Identifier) for the ESKM service.
</li><br><li>
Next, the function uses the get_resource_directory function to obtain the list of available resource instances from the Redfish service. This helps in discovering the necessary URIs.
</li><br><li>
If the DISABLE_RESOURCE_DIR flag is set or there are no resource instances available, an alternative approach is taken to find the relevant URI. The function navigates through the Redfish resource hierarchy to locate the URI for the security service associated with the iLO system.
</li><br><li>
Once the security service URI is obtained, the function retrieves the ESKM URI from the response. This URI points to the specific ESKM resource within the HPE server.
</li><br><li>
If the resource directory is available, the function iterates through the resource instances to find the URI with the identifier '#HpeESKM.' This identifier indicates the presence of the ESKM resource.
</li><br><li>
If a valid ESKM URI is found, the function makes a GET request to retrieve the ESKM data using the Redfish client object (_redfishobj).
</li></ul><br>
Finally, the fetched ESKM data is printed in a nicely formatted JSON representation, providing the necessary information about the ESKM configuration.
 </div>


In [ ]:
def get_ESKM(_redfishobj):

    security_service_eskm_uri = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        security_service_uri = managers_members_response.obj.Oem.Hpe.Links\
                                                                ['SecurityService']['@odata.id']
        security_service_response = _redfishobj.get(security_service_uri)
        security_service_eskm_uri = security_service_response.obj.Links['ESKM']['@odata.id']
    else:
        for instance in resource_instances:
            #Use Resource directory to find the relevant URI
            if '#HpeESKM.' in instance['@odata.type']:
                security_service_eskm_uri = instance['@odata.id']
                break

    if security_service_eskm_uri:
        security_service_eskm_resp = _redfishobj.get(security_service_eskm_uri)
        print(json.dumps(security_service_eskm_resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is the Main function overview.</h3>
    <br>
 <ul><li>It sets up the connection parameters, including the server URL, login credentials, and a flag for enabling the resource directory feature.
</li><br><li>It creates a RedfishClient object using the connection parameters and logs in to the server.
</li><br><li>If the server is unreachable or does not support the Redfish protocol, it displays an error message and exits the script.
</li><br><li>It calls the get_ESKM function to retrieve and print the ESKM data from the HPE iLO system.
</li><br><li>Finally, it logs out from the server, ending the session.
</li></ul><br>In summary, the main function establishes a connection with the HPE server, retrieves ESKM data, and ensures a clean logout from the server.

 </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b> use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br><br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    get_ESKM(REDFISHOBJ)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>